Importando as bibliotecas

In [4]:
# Importando as bibliotecas utilizadas para analisar com mais profundidade a base de dados, utilizando linguagem SQL e Python.
import pandas as pd
import csv

ModuleNotFoundError: No module named 'pandas'

Criação da query para unificar as duas tabelas de receita e despesa, soma do valor total e, por fim agrupamento por fontes de recurso e tipos de despesa.

In [ ]:
# Ler os arquivos CSV e carregar os dados em DataFrames
gdvDespesasExcel = pd.read_csv('/content/gdvDespesasExcel.csv')
gdvReceitasExcel = pd.read_csv('/content/gdvReceitasExcel.csv')

# Consulta
# Combinar os DataFrames usando as colunas 'fonte_recursos' e 'descricao' como chaves de junção
resultado = pd.merge(gdvReceitasExcel, gdvDespesasExcel, 
                     left_on=['fonte_recursos', 'descricao'], 
                     right_on=['fonte_recursos', 'descricao'], 
                     how='outer')

# Preencher os valores ausentes nas colunas 'fonte_recursos', 'descricao' e 'tipo' com valores existentes da mesma coluna
# Isso não terá efeito, pois estamos preenchendo as colunas com elas mesmas (não há valores ausentes nelas)
resultado['fonte_recursos'] = resultado['fonte_recursos'].fillna(resultado['fonte_recursos'])
resultado['descricao'] = resultado['descricao'].fillna(resultado['descricao'])

# Preencher os valores ausentes na coluna 'tipo' com os valores correspondentes na coluna 'tipo_receita' ou 'tipo_despesa'
# Dependendo de onde o valor não estiver ausente
# Isso pode ser útil para garantir que a coluna 'tipo' tenha um valor preenchido, mesmo que a outra tabela não forneça esse valor
resultado['tipo'] = resultado['tipo_receita'].fillna(resultado['tipo_despesa'])


# Converter colunas 'arrecadado' e 'liquidado' para numérico
resultado['arrecadado'] = pd.to_numeric(resultado['arrecadado'], errors='coerce').fillna(0)
resultado['liquidado'] = pd.to_numeric(resultado['liquidado'], errors='coerce').fillna(0)

# Calcular a coluna 'total' e converter para string
resultado['total'] = (resultado['arrecadado'] + resultado['liquidado']).astype(str)

# Reordenar as colunas para exibição
resultado = resultado[['fonte_recursos', 'descricao', 'tipo', 'total']]

# Agrupar e ordenar os resultados
resultado = resultado.groupby(['tipo', 'fonte_recursos', 'descricao']).agg({'total': 'sum'}).reset_index()
resultado = resultado.sort_values(by=['tipo', 'fonte_recursos', 'descricao'])

# Configurar o Pandas para exibir todas as linhas
pd.options.display.max_rows = None

# Exibir todos os dados da tabela como DataFrame
display(resultado)



Fazendo a ingestão do sqlite3 da máquina

In [ ]:
import sqlite3

# Ler o arquivo CSV em um DataFrame
df = pd.read_csv('gdvDespesasExcel.csv')

# Conectar-se ao banco de dados SQLite (cria o banco de dados se não existir)
conn = sqlite3.connect('Teste.db')

# Inserir os dados do DataFrame na tabela do banco de dados SQLite
df.to_sql('gdvDespesas', conn, if_exists='replace', index=False)

# Fechar a conexão com o banco de dados
conn.close()

print("Ingestão de dados concluída com sucesso!")


as 10 maiores fontes de recursos são: tesouro-dot.inicial e cred.suplementar, rec.proprio-adm.ind.-dot.inic.cr.supl., recursos vinculados-fundo especial de despes, recursos vinculados federais, recursos vinculados estaduais, tesouro-dot.inicial e cred.suplementar-intra, rec.prop.adm.ind-cred.p/supervavit financeir, rec.vinc.transf.federal/superavit financ., f.e.d - credito por superavit financeiro, rec.vinc.estaduais-cred.superavit financeiro

os 10 maiores tipos de despesa são: transferencia a municipios-icms, pessoal civil pago pelo ddpe, contrib estado p/cobertura insuf fin-spprev, inativos-rpps, juros sobre a divida por contrato, transferencia a municipios-ipva, pessoal civil pago pela unidade, contribuicoes ao fundeb, contrib patronal-sao paulo previdencia-sppre, contratos de gestao-lei compl.846/98 alt.pos,  vencim., vantag.fixas-pessoal militar